# 定量构效关系(QSAR)模型从0到1 & Uni-Mol入门实践

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：
            <b>
            <a href="mailto:zhengh@dp.tech">郑行 📨 </a>
            </b>
        </i>
        <br/>
        <i>日期：2023-06-06</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i><br/>
        <i>快速开始：点击上方的</i> <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">开始连接</span> <i>按钮，选择 <b><u>unimol-qsar:0703</u>镜像</b>及任意GPU节点配置，稍等片刻即可运行。
    </p>
</div>



近年来，人工智能（AI）正以前所未有的速度发展，为各个领域带来巨大的突破和变革。

而实际上，在药物研发领域，药物科学家从上世纪就开始运用一系列数学和统计方法来助力药物研发的流程。他们基于药物分子的结构，构建数学模型，用以预测药物的生化活性，这种方法被称为**定量构效关系**(Quantitative Structure-Activity Relationship，**QSAR**)。QSAR模型也随着人们对药物分子研究的不断深入，以及更多的人工智能方法被提出而持续发展。

可以说，QSAR模型是一个很好的观察AI for Science领域发展的缩影。在这个Notebook中，我们将以案例的形式介绍不同类型的QSAR模型的构建方法。


## 引言

**定量构效关系**(Quantitative Structure-Activity Relationship，**QSAR**)是一种研究化合物的化学结构与生物活性之间定量关系的方法，是计算机辅助药物设计(Computer-Aided Drug Design, CADD)中最为重要的工具之一。QSAR旨在建立数学模型，构建分子结构与其生化、物化性质关系，帮助药物科学家对新的药物分子的性质开展合理预测。

构建一个有效的QSAR模型涉及到若干步骤：
1. 构建合理的**分子表征**(Molecular Representation)，将分子结构转化为计算机可读的数值表示；
2. 选择适合分子表征的机器学习模型，并使用已有的分子-性质数据**训练模型**；
3. 使用训练好的机器学习模型，对未测定性质的分子进行**性质预测**。

QSAR模型的发展也正是随着**分子表征**的演进，以及对应**机器学习模型**的升级而不断变化。
在这个Notebook中，我们将以案例的形式介绍不同类型的QSAR模型的构建方法。

### 先准备一些数据吧！

为了带领大家更好地学习和体验构建QSAR模型的过程，我们将使用**BACE-1靶点分子活性预测**任务来作为演示案例。


我们可以首先下载BACE-1数据集：

In [3]:
import os
os.makedirs("datasets", exist_ok=True)

!wget https://bohrium-example.oss-cn-zhangjiakou.aliyuncs.com/notebook/bace_classification/train.csv -O ./datasets/BACE_train.csv
!wget https://bohrium-example.oss-cn-zhangjiakou.aliyuncs.com/notebook/bace_classification/test.csv -O ./datasets/BACE_test.csv

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


然后，我们可以观察一下这个数据集的组成：

In [2]:
import pandas as pd

train_data = pd.read_csv("./datasets/BACE_train.csv")
train_data.columns=["SMILES", "TARGET"]
train_data.to_csv("./datasets/BACE_train.csv", index=False)
test_data = pd.read_csv("./datasets/BACE_test.csv")
test_data.columns=["SMILES", "TARGET"]
test_data.to_csv("./datasets/BACE_test.csv", index=False)

print("------ train data ------")
print(train_data)
print("POSITIVE:", sum(train_data["TARGET"]==1))
print("NEGETIVE:", sum(train_data["TARGET"]==0))

print("------ test  data ------")
print(test_data)
print("POSITIVE:", sum(test_data["TARGET"]==1))
print("NEGETIVE:", sum(test_data["TARGET"]==0))

------ train data ------
                                                 SMILES  TARGET
0     CN1C(=O)[C@@](c2ccc(OC(F)F)cc2)(c2ccc(F)c(C#CC...       1
1     CN1C(=O)[C@@](c2ccc(OC(F)F)cc2)(c2cccc(C#CCF)c...       1
2     CN1C(=O)[C@@](c2ccc(OC(F)F)cc2)(c2cccc(C#CCCF)...       1
3     CN1C(=O)[C@@](c2ccc(OC(F)F)cc2)(c2cccc(C#CCCO)...       1
4     CN1C(=O)[C@@](c2ccc(OC(F)F)cc2)(c2cccc(C#CCCCC...       1
...                                                 ...     ...
1205       CN1C(=O)C(c2ccncc2)(c2cccc(-c3ccoc3)c2)N=C1N       0
1206  CC(=O)N[C@@H](Cc1cc(F)cc(F)c1)[C@H](O)C[NH2+]C...       0
1207  O=C(C1C[NH2+]CC1c1ccccc1Br)N1CCC(c2ccccc2)CC1c...       0
1208  Nc1cccc(Cn2c(-c3ccc(Oc4ccncc4)cc3)ccc2-c2ccccc...       0
1209  CC1(C)Cc2cc(Cl)ccc2C(NC(Cc2cscc2CCC2CC2)C(=O)[...       0

[1210 rows x 2 columns]
POSITIVE: 515
NEGETIVE: 695
------ test  data ------
                                                SMILES  TARGET
0    Cc1ccccc1-c1ccc2nc(N)c(CC(C)C(=O)NCC3CCOCC3)cc2c1       1
1  

可以看到，在BACE数据集里：
- 分子用SMILES字符串表示；
- 任务目标是一个二分类任务，其中TARGET==1代表分子对BACE-1靶点具有活性，TARGET==0代表分子对BACE-1靶点没有活性。

这是一个常见的分子性质预测任务。好的，先把这个数据集放一边。接下来，让我们正式进入探索

## QSAR的简明历史

**定量构效关系**(Quantitative Structure-Activity Relationship，**QSAR**)是一种研究化合物的化学结构与生物活性之间定量关系的方法，是计算机辅助药物设计(Computer-Aided Drug Design, **CADD**)中最为重要的工具之一。QSAR旨在建立数学模型，构建分子结构与其生化、物化性质关系，帮助药物科学家对新的药物分子的性质开展合理预测。

QSAR是由**构效关系**（Structure-Activity Relationship，**SAR**）分析发展而来的。SAR的起源可以追溯到19世纪末，当时化学家们开始研究化合物的结构与生物活性之间的关系，德国化学家Paul Ehrlich（1854-1915），他提出了“锁-钥”假说，即化合物（钥匙）与生物靶标（锁）之间的相互作用取决于它们的空间匹配度。随着科学家对分子间相互作用的深入理解，大家发现除了空间匹配外，靶点表面性质（例如疏水性、亲电性）与配体对应结构的性质相互匹配也至关重要，于是发展了一系列评价结构特性与结合亲和力的分析方法，即构效关系。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/56ab433d912a4757a313fbeba2f67b8c/e71248ba-f4bc-4654-a4ac-8bedd211d0b9.png" width="70%" height="70%">

然而，SAR方法主要依赖于化学家的经验和直观判断，缺乏严密的理论基础和统一的分析方法。为了克服这些局限性，20世纪60年代，科学家们开始尝试使用**数学和统计方法**对**分子结构**与**生物活性**之间的关系进行**定量分析**。

最早提出的QSAR模型可以追溯到1868年，化学家Alexander Crum Brown和生理学家Thomas R. Fraser开始研究化合物结构与生物活性之间的关系。在研究生物碱的碱性N原子甲基化前后的生物效应时，他们提出化合物的生理活性依赖于其组分的构成，即生物活性$\phi$是化合物组成$C$的函数：$\phi=f(C)$，这就是著名的**Crum-Brown方程**。这一假设为后来的QSAR研究奠定了基础。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/f0fed56cab8b49e399924799b34409d6/eb0a651d-fa42-45ca-bdf6-18adc77996d3.png" width="70%" height="70%">

随后，不断有QSAR模型在学界被提出，例如Hammett提出的有机物毒性与分子电性的QSAR模型、Taft提出的立体参数模型。1964年，Hansch和Fujita提出了著名的**Hansch模型**，指出分子的生物活性主要是由其疏水效应($\log P$)、立体效应($E_{s}$)和静电效应($\sigma$)决定的，并假设这三种效应彼此可以独立相加，其完整形式为：$\log(1/C)=\alpha\log P + \beta E_{s} + \theta \sigma + \eta$。Hansch模型首次将化学信息与药物生物活性之间的关系进行了定量化描述，为后续的QSAR研究提供了一个实用的理论框架，被认为是从盲目药物设计过渡到合理药物设计的重要标志。

时至今日，QSAR已经发展成为一个成熟的研究领域，涉及多种计算方法和技术。近年来，随着机器学习和人工智能技术的快速发展，QSAR方法得到了进一步的拓展和应用。例如，深度学习技术被应用于QSAR模型的构建，提高了模型的预测能力和准确性。此外，QSAR方法在环境科学、材料科学等领域也取得了广泛的应用，显示出其强大的潜力和广泛的应用前景。


## QSAR建模的基本要求

2002年在葡萄牙的Setubal召开的一次国际会议上，与会的科学工作者们提出了关于QSAR模型有效性的几条规则，被称为“Setubal Principles”，这些规则在 2004年11月得到了进一步详细的修正，并被正式命名为“OECD Principles”。规定一个QSAR模型要达到调控目的（regulatory purpose），应该满足以下5个条件：
1. a defined endpoint（明确目标）
2. an unambiguous algorithm（明确算法）
3. a defined domain of applicability（明确的使用范围）
4. appropriate measures of goodness-of-fit, robustness and predictivity（稳定）
5. a mechanistic interpretation, if possible（如果可能的话，可解释）


## QSAR建模的基本流程

构建一个有效的QSAR模型主要有三步：
1. 构建合理的**分子表征**(Molecular Representation)，将分子结构转化为计算机可读的数值表示；
2. 选择适合分子表征的机器学习模型，并使用已有的分子-性质数据**训练模型**；
3. 使用训练好的机器学习模型，对未测定性质的分子进行**性质预测**。


<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/46ddd8768279413da282a882ec7342ea/63eaf3d4-2ed8-4fcf-b770-21ffdbb3718b.png" width="70%" height="70%">


由于分子结构并不是一个计算机可读的格式，因此我们首先要将分子结构转化为计算机可读的数值向量，才能基于其选择合适的数学模型。我们把这个过程称为**分子表示**(molecular representation)。有效的分子表示以及匹配的数学模型选择是构建定量构效关系模型的核心。

### 分子表示

**分子表示**是包含**分子属性**的数值表示。例如我们常见的分子描述符(Descriptor)、分子指纹(Fingerprints)、SMILES字符串、分子势函数等都是常见的分子表示方法。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/bcb08d76ceb24683959938ed06bb0258/60d68395-1e21-48bf-ab45-8bd10bfbee78.png" width="70%" height="70%">


> Wei, J., Chu, X., Sun, X. Y., Xu, K., Deng, H. X., Chen, J., ... & Lei, M. (2019). Machine learning in materials science. InfoMat, 1(3), 338-358.


事实上，**QSAR的发展也正是随着分子表示包含的信息不断增多、分子表示的形式不断变化而产生分类**，常见的QSAR模型可以分为1D-QSAR、2D-QSAR、3D-QSAR：

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/42f1c735789e4bfbacdf6629f0fd63d3/945acf65-b74a-43de-b6b4-e8d644385fab.png" width="70%" height="70%">

不同的分子表示有不同的数值特点，因此也要选用不同的机器学习/深度学习模型进行建模。接下来，我们就将以实际案例给大家展示如何构建1D-QSAR, 2D-QSAR, 3D-QSAR模型。

### 1D-QSAR分子表征

早期的定量构效关系模型大多以分子量、水溶性、分子表面积等分子的物化性质作为表征的方法，我们称这些分子的物化性质为分子描述符（Descriptor）。这就是1D-QSAR的阶段。

这个阶段往往需要富有经验的科学家基于自己的领域知识，来进行分子描述符的设计，去构建一些可能和这个性质相关的一些分子性质。例如假设要预测某个药物是否能通过血脑屏障，那这个性质可能和药物分子的水溶性、分子量、极性表面积等物化属性相关，科学家就要把这样的属性加入到分子描述符中。

这个阶段由于计算机尚未普及，或算力不足，科学家往往通过一些简单的数学模型进行建模，例如线性回归、随机森林等方法。当然了，由于通过分子描述符构建的分子表示通常是低维的实值向量，这些数学模型也很适合做这样的工作。

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def calculate_1dqsar_repr(smiles):
    # 从 SMILES 字符串创建分子对象
    mol = Chem.MolFromSmiles(smiles)
    # 计算分子的分子量
    mol_weight = Descriptors.MolWt(mol)
    # 计算分子的 LogP 值
    log_p = Descriptors.MolLogP(mol)
    # 计算分子中的氢键供体数量
    num_h_donors = Descriptors.NumHDonors(mol)
    # 计算分子中的氢键受体数量
    num_h_acceptors = Descriptors.NumHAcceptors(mol)
    # 计算分子的表面积极性
    tpsa = Descriptors.TPSA(mol)
    # 计算分子中的可旋转键数量
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 计算分子中的芳香环数量
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    # 计算分子中的脂环数量
    num_aliphatic_rings = Descriptors.NumAliphaticRings(mol)
    # 计算分子中的饱和环数量
    num_saturated_rings = Descriptors.NumSaturatedRings(mol)
    # 计算分子中的杂原子数量
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 计算分子中的价电子数量
    num_valence_electrons = Descriptors.NumValenceElectrons(mol)
    # 计算分子中的自由基电子数量
    num_radical_electrons = Descriptors.NumRadicalElectrons(mol)
    # 计算分子的 QED 值
    qed = Descriptors.qed(mol)
    # 返回所有计算出的属性值
    return [mol_weight, log_p, num_h_donors, num_h_acceptors, tpsa, num_rotatable_bonds, num_aromatic_rings,
            num_aliphatic_rings, num_saturated_rings, num_heteroatoms, num_valence_electrons, num_radical_electrons,qed]


train_data["1dqsar_mr"] = train_data["SMILES"].apply(calculate_1dqsar_repr)
test_data["1dqsar_mr"] = test_data["SMILES"].apply(calculate_1dqsar_repr)

In [4]:
print(train_data["1dqsar_mr"][0])

[433.40500000000026, 3.558600000000002, 1, 4, 67.92, 6, 2, 1, 0, 9, 162, 0, 0.4304551686487377]


In [5]:
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# 将训练和测试数据转换为NumPy数组
train_x = np.array(train_data["1dqsar_mr"].values.tolist())
train_y = np.array(train_data["TARGET"].values.tolist())
test_x = np.array(test_data["1dqsar_mr"].values.tolist())
test_y = np.array(test_data["TARGET"].values.tolist())

# 定义要使用的分类器列表
classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("Stochastic Gradient Descent", SGDClassifier(loss='log',random_state=42)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Bernoulli Naive Bayes", BernoulliNB()),
    ("Decision Tree", DecisionTreeClassifier(random_state=42)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
    ("Multi-layer Perceptron", MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=10000, random_state=42)),
]

# 初始化结果字典
results = {}

# 对每个分类器进行训练和预测，并计算各项性能指标
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    pred_y = classifier.predict(test_x)
    pred_y_proba = classifier.predict_proba(test_x)[:, 1]
    accuracy = accuracy_score(test_y, pred_y)
    auc = roc_auc_score(test_y, pred_y_proba)
    fpr, tpr, _ = roc_curve(test_y, pred_y_proba)
    results[f"1D-QSAR-{name}"] = {"ACC": accuracy, "AUC": auc, "FPR": fpr, "TPR": tpr}
    print(f"[1D-QSAR][{name}]\tACC:{accuracy:.4f}\tAUC:{auc:.4f}")

# 根据AUC对结果进行排序
sorted_results = sorted(results.items(), key=lambda x: x[1]["AUC"], reverse=True)

# 绘制ROC曲线
plt.figure(figsize=(10, 6), dpi=200)
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for name, result in sorted_results:
    if name.startswith("1D-QSAR"):
        plt.plot(result["FPR"], result["TPR"], label=f"{name} (ACC:{result['ACC']:.4f} AUC:{result['AUC']:.4f})")

plt.legend(loc="lower right")
plt.show()

/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[1D-QSAR][Logistic Regression]	ACC:0.5644	AUC:0.6396
[1D-QSAR][Stochastic Gradient Descent]	ACC:0.4191	AUC:0.5028
[1D-QSAR][K-Nearest Neighbors]	ACC:0.6040	AUC:0.6287
[1D-QSAR][Bernoulli Naive Bayes]	ACC:0.4092	AUC:0.4571
[1D-QSAR][Decision Tree]	ACC:0.5842	AUC:0.5964
[1D-QSAR][Random Forest]	ACC:0.6172	AUC:0.6790
[1D-QSAR][XGBoost]	ACC:0.6502	AUC:0.6832
[1D-QSAR][Multi-layer Perceptron]	ACC:0.5710	AUC:0.6488


<Figure size 2000x1200 with 1 Axes>

### 2D-QSAR分子表征

然而，面临一些生化机制尚不清晰的分子性质预测问题时，科学家可能很难设计出有效的分子描述符来表征分子，导致QSAR模型构建的失败。由于分子性质很大程度由分子结构决定，例如分子上有什么官能团，因此人们想把分子的键连关系引入到QSAR建模中。于是领域进入了2D-QSAR的阶段。

较早被提出的是Morgan指纹等通过遍历分子中每个原子与周边原子的键连关系来表征的分子指纹方法。为了满足不同大小的分子能用相同长度的数值向量来表征的要求，分子指纹往往会通过hash的操作来保证向量长度的统一，因此分子指纹往往是高维的0/1向量。在这个场景下，科学家通常会选择例如支持向量机，以及全连接神经网络等对高维稀疏向量有较好处理能力的机器学习方法来进行模型构建。

随着AI模型的发展，能处理序列数据（例如文本）的循环神经网络（Recurrent neural network, RNN）、能处理图片数据的卷积神经网络（convolutional neural network, CNN）、能处理非结构化的图数据的图神经网络（graph neural network, GNN）等深度学习模型不断被提出和应用，QSAR模型也根据这些模型能处理的数据特点，构建了适配的分子表示。例如人们将分子的SMILES字符表示应用RNN建模，将分子的二维图像应用CNN建模，将分子的键连拓扑结构转化成图应用GNN建模发展了一系列的QSAR建模方法。

但是总的来说，在2D-QSAR阶段中，人们在利用各类方法去解析分子的键连关系（拓扑结构）来进行分子性质的建模预测。

In [6]:
import numpy as np
from rdkit.Chem import AllChem

def calculate_2dqsar_repr(smiles):    
    # 将smiles字符串转换为rdkit的分子对象
    mol = Chem.MolFromSmiles(smiles)
    # 计算Morgan指纹（半径为3，长度为1024位）
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024)
    # 返回numpy数组格式的指纹
    return np.array(fp)

train_data["2dqsar_mr"] = train_data["SMILES"].apply(calculate_2dqsar_repr) 
test_data["2dqsar_mr"] = test_data["SMILES"].apply(calculate_2dqsar_repr) 

In [7]:
print(train_data["2dqsar_mr"][0].tolist())

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# 将训练和测试数据转换为NumPy数组
train_x = np.array(train_data["2dqsar_mr"].values.tolist())
train_y = np.array(train_data["TARGET"].values.tolist())
test_x = np.array(test_data["2dqsar_mr"].values.tolist())
test_y = np.array(test_data["TARGET"].values.tolist())

# 定义要使用的分类器列表
classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("Stochastic Gradient Descent", SGDClassifier(loss='log',random_state=42)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Bernoulli Naive Bayes", BernoulliNB()),
    ("Decision Tree", DecisionTreeClassifier(random_state=42)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
    ("Multi-layer Perceptron", MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=10000, random_state=42)),
]


# 对每个分类器进行训练和预测，并计算各项性能指标
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    pred_y = classifier.predict(test_x)
    pred_y_proba = classifier.predict_proba(test_x)[:, 1]
    accuracy = accuracy_score(test_y, pred_y)
    auc = roc_auc_score(test_y, pred_y_proba)
    fpr, tpr, _ = roc_curve(test_y, pred_y_proba)
    results[f"2D-QSAR-{name}"] = {"ACC": accuracy, "AUC": auc, "FPR": fpr, "TPR": tpr}
    print(f"[2D-QSAR][{name}]\tACC:{accuracy:.4f}\tAUC:{auc:.4f}")

# 根据AUC对结果进行排序
sorted_results = sorted(results.items(), key=lambda x: x[1]["AUC"], reverse=True)

# 绘制ROC曲线
plt.figure(figsize=(10, 6), dpi=200)
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for name, result in sorted_results:
    if name.startswith("2D-QSAR"):
        plt.plot(result["FPR"], result["TPR"], label=f"{name} (ACC:{result['ACC']:.4f} AUC:{result['AUC']:.4f})")

plt.legend(loc="lower right")
plt.show()

[2D-QSAR][Logistic Regression]	ACC:0.6799	AUC:0.7582


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[2D-QSAR][Stochastic Gradient Descent]	ACC:0.6865	AUC:0.7424
[2D-QSAR][K-Nearest Neighbors]	ACC:0.7162	AUC:0.7419
[2D-QSAR][Bernoulli Naive Bayes]	ACC:0.6568	AUC:0.7329
[2D-QSAR][Decision Tree]	ACC:0.6139	AUC:0.6369
[2D-QSAR][Random Forest]	ACC:0.6601	AUC:0.7660
[2D-QSAR][XGBoost]	ACC:0.6733	AUC:0.7529
[2D-QSAR][Multi-layer Perceptron]	ACC:0.6601	AUC:0.7138


<Figure size 2000x1200 with 1 Axes>

### 3D-QSAR分子表征

然而，由于分子间、分子内相互作用的存在，拓扑结构相近的分子在各个不同环境下会采取的构象不尽相同。而每个分子在不同环境下的构象以及对应的能量高低决定了分子的真实性质。因此，科学家期望将分子的三维结构引入到QSAR建模里去，来增强对特定场景的分子性质预测能力。这个阶段被称为3D-QSAR阶段。

分子比较场方法（CoFMA）是被广泛应用的3D-QSAR模型。它计算分子存在的空间中各个位置（通常通过格点法进行位置的选取）所受到的力的作用（也就是力场）来表征分子的三维结构。当然，领域中还有一些有益的尝试，包括通过电子密度、分子三维图像等表征方法，或是在分子图上加入几何信息。

而要处理这样的高维空间信息，科学家们往往会选择例如较深的FCNN、3D-CNN、GNN等深度学习方法来进行建模。

In [9]:
from rdkit.Chem import rdPartialCharges

def calculate_3dqsar_repr(SMILES, max_atoms=100, three_d=False):
    mol = Chem.MolFromSmiles(SMILES)  # 从SMILES表示创建分子对象
    mol = Chem.AddHs(mol)  # 添加氢原子
    if three_d:
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # 计算3D坐标
    else:
        AllChem.Compute2DCoords(mol)  # 计算2D坐标
    natoms = mol.GetNumAtoms()  # 获取原子数量
    rdPartialCharges.ComputeGasteigerCharges(mol)  # 计算分子的Gasteiger电荷
    charges = np.array([float(atom.GetProp("_GasteigerCharge")) for atom in mol.GetAtoms()])  # 获取电荷值
    coords = mol.GetConformer().GetPositions()  # 获取原子坐标
    coulomb_matrix = np.zeros((max_atoms, max_atoms))  # 初始化库仑矩阵
    n = min(max_atoms, natoms)
    for i in range(n):  # 遍历原子
        for j in range(i, n):
            if i == j:
                coulomb_matrix[i, j] = 0.5 * charges[i] ** 2
            if i != j:
                delta = np.linalg.norm(coords[i] - coords[j])  # 计算原子间距离
                if delta != 0:
                    coulomb_matrix[i, j] = charges[i] * charges[j] / delta  # 计算库仑矩阵的元素值
                    coulomb_matrix[j, i] = coulomb_matrix[i, j]
    coulomb_matrix = np.where(np.isinf(coulomb_matrix), 0, coulomb_matrix)  # 处理无穷大值
    coulomb_matrix = np.where(np.isnan(coulomb_matrix), 0, coulomb_matrix)  # 处理NaN值
    return coulomb_matrix.reshape(max_atoms*max_atoms).tolist()  # 将库仑矩阵转换为列表并返回


train_data["3dqsar_mr"] = train_data["SMILES"].apply(calculate_3dqsar_repr) 
test_data["3dqsar_mr"] = test_data["SMILES"].apply(calculate_3dqsar_repr) 

In [10]:
print(len(train_data.iloc[0]["3dqsar_mr"]))

10000


In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# 将训练和测试数据转换为NumPy数组
train_x = np.array(train_data["3dqsar_mr"].values.tolist())
train_y = np.array(train_data["TARGET"].values.tolist())
test_x = np.array(test_data["3dqsar_mr"].values.tolist())
test_y = np.array(test_data["TARGET"].values.tolist())

# 定义要使用的分类器列表
classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("Stochastic Gradient Descent", SGDClassifier(loss='log',random_state=42)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Bernoulli Naive Bayes", BernoulliNB()),
    ("Decision Tree", DecisionTreeClassifier(random_state=42)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
    ("Multi-layer Perceptron", MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', max_iter=10000, random_state=42)),
]


# 对每个分类器进行训练和预测，并计算各项性能指标
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    pred_y = classifier.predict(test_x)
    pred_y_proba = classifier.predict_proba(test_x)[:, 1]
    accuracy = accuracy_score(test_y, pred_y)
    auc = roc_auc_score(test_y, pred_y_proba)
    fpr, tpr, _ = roc_curve(test_y, pred_y_proba)
    results[f"3D-QSAR-{name}"] = {"ACC": accuracy, "AUC": auc, "FPR": fpr, "TPR": tpr}
    print(f"[3D-QSAR][{name}]\tACC:{accuracy:.4f}\tAUC:{auc:.4f}")

# 根据AUC对结果进行排序
sorted_results = sorted(results.items(), key=lambda x: x[1]["AUC"], reverse=True)

# 绘制ROC曲线
plt.figure(figsize=(10, 6), dpi=200)
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for name, result in sorted_results:
    if name.startswith("3D-QSAR"):
        plt.plot(result["FPR"], result["TPR"], label=f"{name} (ACC:{result['ACC']:.4f} AUC:{result['AUC']:.4f})")

plt.legend(loc="lower right")
plt.show()

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[3D-QSAR][Logistic Regression]	ACC:0.4191	AUC:0.5000


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[3D-QSAR][Stochastic Gradient Descent]	ACC:0.3762	AUC:0.3785
[3D-QSAR][K-Nearest Neighbors]	ACC:0.6040	AUC:0.7008
[3D-QSAR][Bernoulli Naive Bayes]	ACC:0.6568	AUC:0.6658
[3D-QSAR][Decision Tree]	ACC:0.5974	AUC:0.5906
[3D-QSAR][Random Forest]	ACC:0.6601	AUC:0.7395
[3D-QSAR][XGBoost]	ACC:0.6436	AUC:0.7290
[3D-QSAR][Multi-layer Perceptron]	ACC:0.6304	AUC:0.6524


<Figure size 2000x1200 with 1 Axes>

## Uni-Mol 分子表示学习和预训练框架

### 预训练模型
在药物研发领域中，QSAR建模面临的一个主要挑战是数据量有限。由于药物活性数据的获取成本高且实验难度大，这导致了**标签数据不足**的情况。数据量不足会影响模型的预测能力，因为模型可能难以捕捉到足够的信息来描述化合物结构和生物活性之间的关系。

面临这种有标签数据不足的情况，在机器学习发展地更为成熟的领域，例如自然语言处理(NLP)和计算机视觉(CV)中，**预训练-微调(Pretrain-Finetune)模式**已经成为了通用的解决方案。预训练是指在大量无标签数据对模型通过自监督学习进行预先训练，使模型获得一些基本信息和通用能力，然后再在有限的有标签数据上进行监督学习来微调模型，使模型在具体问题上具备特定问题的推理能力。

例如，我想进行猫狗的图片识别，但是我没有很多猫狗的有标签数据。于是我可以先用大量的没有标签的图片预训练模型，先让模型学到点线面轮廓的基本知识，然后再把猫狗图片给模型做有监督训练，这时候，模型可能就能基于轮廓信息，快速学习到什么是猫什么是狗的信息了。

预训练方法可以充分利用大量容易获取的无标签数据的信息，提高模型的泛化能力和预测性能。在QSAR建模中，我们同样可以借鉴预训练的思想来解决数据数量和数据质量问题。

### Uni-Mol 简介

Uni-Mol是深势科技于2022年5月发布的一款基于分子三维结构的通用分子表征学习框架。Uni-Mol将**分子三维结构作为模型输入**，并使用**约2亿个小分子构象**和**300万个蛋白表面空腔结构**，使用**原子类型还原**和**原子坐标还原**两种自监督任务对模型进行**预训练**。

> Uni-Mol 论文：https://openreview.net/forum?id=6K2RM6wVqKu<br>开源代码：https://github.com/dptech-corp/Uni-Mol 


从三维信息出发的表征学习和有效的预训练方案让 Uni-Mol 在几乎所有与药物分子和蛋白口袋相关的下游任务上都超越了 SOTA（state of the art），也让 Uni-Mol 得以能够直接完成分子构象生成、蛋白-配体结合构象预测等三维构象生成相关的任务，并超越现有解决方案。论文被机器学习顶会ICLR 2023接收。


<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/110155/19d4423c3c5242fe98a4464276377246/d592ed9f-78e4-4d51-84d4-44a2f88e95f3.png" width="70%" height="70%">

接下来，我们要使用Uni-Mol来完成BACE-1分子活性预测任务的构建：

In [12]:
from unimol import MolTrain, MolPredict

clf = MolTrain(task='classification',
                data_type='molecule',
                epochs=100,
                learning_rate=0.00008,
                batch_size=64,
                early_stopping=10,
                metrics='auc',
                split='random',
                save_path='./exp',
              )

clf.fit('datasets/BACE_train.csv')

2023-06-12 14:58:27 | unimol/data/conformer.py | 56 | INFO | Uni-Mol(QSAR) | Start generating conformers...
1210it [00:35, 34.35it/s]
2023-06-12 14:59:02 | unimol/data/conformer.py | 60 | INFO | Uni-Mol(QSAR) | Failed to generate conformers for 0.00% of molecules.
2023-06-12 14:59:02 | unimol/data/conformer.py | 62 | INFO | Uni-Mol(QSAR) | Failed to generate 3d conformers for 0.00% of molecules.
2023-06-12 14:59:02 | unimol/train.py | 83 | INFO | Uni-Mol(QSAR) | Create output directory: ./exp
2023-06-12 14:59:03 | unimol/models/unimol.py | 107 | INFO | Uni-Mol(QSAR) | Loading pretrained weights from /opt/conda/lib/python3.8/site-packages/unimol-0.0.2-py3.8.egg/unimol/weights/mol_pre_all_h_220816.pt
2023-06-12 14:59:03 | unimol/models/nnmodel.py | 100 | INFO | Uni-Mol(QSAR) | start training Uni-Mol:unimolv1
2023-06-12 14:59:11 | unimol/tasks/trainer.py | 156 | INFO | Uni-Mol(QSAR) | Epoch [1/100] train_loss: 0.6934, val_loss: 0.6591, val_auc: 0.7798, lr: 0.000033, 8.1s
2023-06-12 14:59:

In [13]:
predm = MolPredict(load_model='./exp')
test_pred = predm.predict('datasets/BACE_test.csv')

2023-06-12 15:05:01 | unimol/data/conformer.py | 56 | INFO | Uni-Mol(QSAR) | Start generating conformers...
303it [00:12, 25.08it/s]
2023-06-12 15:05:13 | unimol/data/conformer.py | 60 | INFO | Uni-Mol(QSAR) | Failed to generate conformers for 0.00% of molecules.
2023-06-12 15:05:13 | unimol/data/conformer.py | 62 | INFO | Uni-Mol(QSAR) | Failed to generate 3d conformers for 0.00% of molecules.
2023-06-12 15:05:14 | unimol/models/unimol.py | 107 | INFO | Uni-Mol(QSAR) | Loading pretrained weights from /opt/conda/lib/python3.8/site-packages/unimol-0.0.2-py3.8.egg/unimol/weights/mol_pre_all_h_220816.pt
2023-06-12 15:05:14 | unimol/models/nnmodel.py | 145 | INFO | Uni-Mol(QSAR) | start predict NNModel:unimolv1
2023-06-12 15:05:14 | unimol/tasks/trainer.py | 197 | INFO | Uni-Mol(QSAR) | load model success!
2023-06-12 15:05:15 | unimol/tasks/trainer.py | 197 | INFO | Uni-Mol(QSAR) | load model success!
2023-06-12 15:05:16 | unimol/tasks/trainer.py | 197 | INFO | Uni-Mol(QSAR) | load model s

In [14]:
unimol_results = pd.DataFrame({'pred':test_pred.reshape(-1), 
                           'smiles':test_data['SMILES'], 
                           'target':test_data['TARGET']})

plt.figure(figsize=(10, 6), dpi=200)
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

accuracy = accuracy_score(
    unimol_results.target, 
    [1 if p > 0.5 else 0 for p in unimol_results.pred]
)
auc = roc_auc_score(unimol_results.target, unimol_results.pred)
fpr, tpr, _ = roc_curve(unimol_results.target, unimol_results.pred)
plt.plot(fpr, tpr, label=f"Uni-Mol (ACC:{accuracy:.4f} AUC: {auc:.4f})")
print(f"[Uni-Mol]\tACC:{accuracy:.4f}\t AUC:{auc:.4f}")

results["Uni-Mol"] = {"ACC": accuracy, "AUC": auc, "FPR":fpr, "TPR":tpr}
    
plt.legend(loc="lower right")
plt.show()

[Uni-Mol]	ACC:0.6733	 AUC:0.7685


<Figure size 2000x1200 with 1 Axes>

## 结果总览

最后，我们可以横向比较一下1D-QSAR, 2D-QSAR, 3D-QSAR和不同的模型组合，以及Uni-Mol在同一个数据集上的预测表现。

In [17]:
import pandas as pd

df = pd.DataFrame(results).T.drop(columns=["FPR", "TPR"])
df.sort_values(by="AUC", ascending=False, inplace=True)
df

,ACC,AUC
Uni-Mol,0.673267,0.768477
2D-QSAR-Random Forest,0.660066,0.765994
2D-QSAR-Logistic Regression,0.679868,0.758187
2D-QSAR-XGBoost,0.673267,0.75293
2D-QSAR-Stochastic Gradient Descent,0.686469,0.742439
2D-QSAR-K-Nearest Neighbors,0.716172,0.741902
3D-QSAR-Random Forest,0.660066,0.739531
2D-QSAR-Bernoulli Naive Bayes,0.656766,0.73291
3D-QSAR-XGBoost,0.643564,0.729018
2D-QSAR-Multi-layer Perceptron,0.660066,0.713762
